In [12]:
import os
from dotenv import load_dotenv
load_dotenv()

True

## What is RAG (retrieval augmented generation)?
Basically, shoving lot of extra information in the prompt.

In [2]:
# Example 

from langchain.chat_models import init_chat_model
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

response = model.invoke(
    prompt_template.format(
        context="Ajit has two sisters, Preeti and Sweta. Ajit is male.",
        question="Who is Preeti's bother?"         
    ))

print(response)

content="Ajit is Preeti's brother. He is also the brother of Sweta. This is stated in the context where it mentions Ajit has two sisters, Preeti and Sweta, implying Ajit is their male sibling." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 119, 'total_tokens': 168, 'completion_time': 0.18517725, 'prompt_time': 0.020291062, 'queue_time': 0.046105688, 'total_time': 0.205468312}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--fe314a4c-bff8-4d2a-895e-5ba5dc0cb5ae-0' usage_metadata={'input_tokens': 119, 'output_tokens': 49, 'total_tokens': 168}


In [3]:
print(response.content)

Ajit is Preeti's brother. He is also the brother of Sweta. This is stated in the context where it mentions Ajit has two sisters, Preeti and Sweta, implying Ajit is their male sibling.


>RAG is all about cleverly pushing is as much information in the context with minimum possible tokens

In [4]:
# little non-trival RAG example

# grab a novel
file = open("indianTales.txt", encoding="utf-8")
text = file.read()

In [5]:
# lets try to talk with this book

from langchain.chat_models import init_chat_model
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

# Prepare your prompt
prompt_template = """You are a novel reader. You are given collection of stories:
{collection_of_stories}
You are tasked to make a list of story titles in this collection. Write a short summary for each story in Hindi Language. Skip the story from the list, if the story is not provided in the text. 
"""

response = model.invoke(prompt_template.format(collection_of_stories = text[:len(text)//20]))

print(response)

content='यहाँ कहानियों की सूची दी गई है, जिनमें से प्रत्येक का एक छोटा सारांश हिंदी में दिया गया है:\n\n1. **शेर और बगुला**: इस कहानी में, एक शेर के गले में एक हड्डी फंस जाती है, और एक बगुला उसे निकालने में मदद करता है। बाद में, शेर बगुला को धन्यवाद देने के बजाय उसे मारने की कोशिश करता है, लेकिन बगुला बच निकलता है और शेर को सबक सिखाता है।\n\n2. **राजकुमार और राजकुमारी लबाम**: इस कहानी में, एक राजकुमार अपनी माँ की सलाह के विरुद्ध जाकर राजकुमारी लबाम को ढूंढने निकलता है। वह कई चुनौतियों का सामना करता है और अंत में राजकुमारी से मिलता है।' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 275, 'prompt_tokens': 5232, 'total_tokens': 5507, 'completion_time': 0.865247934, 'prompt_time': 0.396021271, 'queue_time': 0.048680449, 'total_time': 1.261269205}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--d4f918dd-01a4-41d8-8115-9b7c9d40e3e0-0' usage_metadata=

In [6]:
# Better printing

from IPython.display import Markdown
Markdown(response.content)

यहाँ कहानियों की सूची दी गई है, जिनमें से प्रत्येक का एक छोटा सारांश हिंदी में दिया गया है:

1. **शेर और बगुला**: इस कहानी में, एक शेर के गले में एक हड्डी फंस जाती है, और एक बगुला उसे निकालने में मदद करता है। बाद में, शेर बगुला को धन्यवाद देने के बजाय उसे मारने की कोशिश करता है, लेकिन बगुला बच निकलता है और शेर को सबक सिखाता है।

2. **राजकुमार और राजकुमारी लबाम**: इस कहानी में, एक राजकुमार अपनी माँ की सलाह के विरुद्ध जाकर राजकुमारी लबाम को ढूंढने निकलता है। वह कई चुनौतियों का सामना करता है और अंत में राजकुमारी से मिलता है।

### Problem: Too much raw information in the context makes the prompt too long.
- Costly
- adds noise
### Solution: Use RAG
https://python.langchain.com/docs/tutorials/rag/
### To understand RAG, we need to understand Semantic Search
https://python.langchain.com/docs/tutorials/retrievers/

In [7]:
# load text using RELEVANT loader
from langchain_community.document_loaders import TextLoader
loader = TextLoader("indianTales.txt")
docs = loader.load()

In [8]:
# Split document into small chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split given book into {len(all_splits)} sub-documents.")

Split given book into 563 sub-documents.


### Embedding

In [13]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create a vector store
# (CLASSROOM DISCUSSION: What are vector stores? What do we make them?)
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

# Adding documents to vector store
document_ids = vector_store.add_documents(documents=all_splits)


GoogleGenerativeAIError: Error embedding content: 400 API key expired. Please renew the API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key expired. Please renew the API key."
]

In [ ]:
# document_ids

In [ ]:
# extract chunks which matches with your query

search_results = vector_store.similarity_search_with_score(
    "What is the role of Lion in the story?",
    k = 10
)

In [ ]:
search_results

### What is RAG?
Retrieve using semantic search and dump the similar chunks in the context of the prompt.
LLM sees the question and retrieved docs in its prompt and generates tokens accordingly.

In [ ]:
prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

In [ ]:
doc_content = "\n\n".join(doc.page_content+"\n"+"="*50+"\n" for (doc,score) in search_results)
print(doc_content)

In [ ]:
# make the LLM read see the prompt, and analyse the retrieved document, and generate response

from langchain.chat_models import init_chat_model
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

response = model.invoke(prompt_template.format(
    context=doc_content,
    question="What is the role of Lion in the story?"))

In [ ]:
# Better printing

from IPython.display import Markdown
Markdown(response.content)

# RAG Summary

In [ ]:
# RAG summary

# Read a doc
from langchain_community.document_loaders import TextLoader
loader = TextLoader("indianTales.txt")
docs = loader.load()

# Split document into small chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split given book into {len(all_splits)} sub-documents.")

# embedding
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create a vector store
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

# Adding documents to vector store
document_ids = vector_store.add_documents(documents=all_splits)


In [ ]:
# extract chunks which matches with your query

search_results = vector_store.similarity_search_with_score(
    "What is the role of Lion in the story?",
    k = 10
)

doc_content = "\n\n".join(doc.page_content+"\n"+"="*50+"\n" for (doc,score) in search_results)
print(doc_content)

In [ ]:
prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

response = model.invoke(prompt_template.format(
    context=doc_content,
    question="What is the role of Lion in the story?"))


from IPython.display import Markdown
Markdown(response.content)